In [1]:
import json
from tqdm import tqdm
import re

def split_text_preserve_code(text):
    code_block_pattern = r"```.*?```"
    code_blocks = re.findall(code_block_pattern, text, re.DOTALL)
    placeholder = "__CODE_BLOCK__"
    text_without_code = re.sub(code_block_pattern, placeholder, text, flags=re.DOTALL)
    parts = text_without_code.split('\n\n')
    result = []
    code_index = 0
    for part in parts:
        if placeholder in part:
            result.append(part.replace(placeholder, code_blocks[code_index], 1))
            code_index += 1
        else:
            result.append(part)
    
    return result

def contains_chinese(text):
    chinese_pattern = re.compile(r'[\u4e00-\u9fff]')
    return bool(chinese_pattern.search(text))

filtered = []
with open('qwq_500k.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        
        if contains_chinese(l['response']):
            continue
            
        r = l['response'].lower()
            
        if 'can\'t perform' in r:
            continue
        
        if 'can\'t answer' in r:
            continue
        
        if len(l['response'].split()) > 1000:
            filtered.append(l)

498082it [01:30, 5477.00it/s] 


In [2]:
len(filtered)

343207

In [4]:
selected = []
for f in filtered:
    if 'wrong' in f['prompt'].lower():
        continue
    if 'ath problem: \n\n' in f['prompt']:
        f['prompt'] = f['prompt'].split('ath problem: \n\n')[1]
    if 'ath problem:\n\n' in f['prompt']:
        f['prompt'] = f['prompt'].split('ath problem:\n\n')[1]
    selected.append(f)

In [5]:
len(filtered), len(selected)

(343207, 341463)

In [6]:
no_code = []
for f in selected:
    if '```' not in f['response']:
        no_code.append(f)
        
len(no_code)

262556

In [12]:
with open('filter-qwq-500k-no-code.jsonl', 'w') as fopen:
    for f in no_code:
        fopen.write(f'{json.dumps(f)}\n')
        fopen.flush()

In [13]:
with open('filter-qwq-500k.jsonl', 'w') as fopen:
    for f in selected:
        fopen.write(f'{json.dumps(f)}\n')
        fopen.flush()